In [2]:
import numpy as np
from cmath import log
import math

In [10]:
#sample data
expected = np.random.normal(100, 20, 10)
poissons = np.vectorize(lambda x: np.random.poisson(3*x))
observed = poissons(expected)

In [11]:
def llr_i(obs, exp, q):
    # q = obs/exp
    return obs*log(q) + exp*(1-q)

llr_i_vec = np.vectorize(llr_i)

scores = llr_i_vec(observed, expected, q=(observed/expected))
score = sum(scores)
score

(1275.1129553606736+0j)

In [5]:
#binary search for q min
def find_q_min(observed, expected):
    low = 0.0000001
    high = observed/expected

    while np.abs(high - low) > 0.000001:
        mid = (low + high) / 2
        
        if (observed*log(mid) + expected*(1-mid)) > 0:
            high = mid
        else:
            low = mid
    return (high+low)/2

def find_q_max(observed, expected):
    low = observed/expected
    high = 1000000
    while np.abs(high - low) > 0.000001:
        mid = (low + high) / 2
        
        if (observed*log(mid) + expected*(1-mid)) > 0:
            high = mid
        else:
            low = mid
    return (high+low)/2

In [6]:
def binary_search_on_score_for_q_min(count,baseline,q_mle):
    q_temp_min = 0.000001
    q_temp_max = q_mle
    while np.abs(q_temp_max-q_temp_min) > 0.000001:
        q_temp_mid = (q_temp_min+q_temp_max)/2
        if np.sign(llr_i(count,baseline, q_temp_mid)) > 0:
            q_temp_max = q_temp_max-(q_temp_max-q_temp_min)/2
        else:
            q_temp_min = q_temp_min+(q_temp_max-q_temp_min)/2
    return (q_temp_min+q_temp_max)/2

def binary_search_on_score_for_q_max(count,baseline,q_mle):
    q_temp_min = q_mle
    q_temp_max = 1000000.0
    while np.abs(q_temp_max-q_temp_min) > 0.000001:
        q_temp_mid = (q_temp_min+q_temp_max)/2
        if np.sign(llr_i(count,baseline, q_temp_mid)) > 0:
            q_temp_min = q_temp_min+(q_temp_max-q_temp_min)/2
        else:
             q_temp_max = q_temp_max-(q_temp_max-q_temp_min)/2
    return (q_temp_min+q_temp_max)/2


def binary_search_on_score_for_q_min_pen(count,baseline,delta,q_mle):
    q_temp_min = 0.000001
    q_temp_max = q_mle
    while np.abs(q_temp_max-q_temp_min) > 0.000001:
        q_temp_mid = (q_temp_min+q_temp_max)/2
        if np.sign(llr_i(count,baseline, q_temp_mid) - delta) > 0:
            q_temp_max = q_temp_max-(q_temp_max-q_temp_min)/2
        else:
            q_temp_min = q_temp_min+(q_temp_max-q_temp_min)/2
    return (q_temp_min+q_temp_max)/2

def binary_search_on_score_for_q_max_pen(count,baseline,delta,q_mle):
    q_temp_min = q_mle
    q_temp_max = 1000000.0
    while np.abs(q_temp_max-q_temp_min) > 0.000001:
        q_temp_mid = (q_temp_min+q_temp_max)/2
        if np.sign(llr_i(count,baseline,q_temp_mid) - delta) > 0: 
            q_temp_min = q_temp_min+(q_temp_max-q_temp_min)/2
        else:
             q_temp_max = q_temp_max-(q_temp_max-q_temp_min)/2
    return (q_temp_min+q_temp_max)/2

In [7]:
ct = observed[1]
bs = expected[1]
qmle_array = observed/expected
qmle_array[1]

delta = 0.01

print(binary_search_on_score_for_q_min(ct, bs, qmle))
print(binary_search_on_score_for_q_min_pen(ct, bs, delta, qmle))
print(binary_search_on_score_for_q_max_pen(ct, bs, delta, qmle))
print(binary_search_on_score_for_q_max(ct, bs, qmle))

def find_interval(observed, expected, delta, qmle):
    min_unpen = binary_search_on_score_for_q_min(observed, expected, qmle)
    min_pen = binary_search_on_score_for_q_min_pen(observed, expected, delta, qmle)
    max_pen = binary_search_on_score_for_q_max_pen(observed, expected, delta, qmle)
    max_unpen = binary_search_on_score_for_q_max(observed, expected, qmle)
    return [min_unpen, min_pen, max_pen, max_unpen]

find_intervals = np.vectorize(find_interval)

NameError: name 'qmle' is not defined

In [9]:
intervals = [[0]*4 for i in range(len(observed))]

for i in range(len(observed)):
    min_unpen = binary_search_on_score_for_q_min(observed[i], expected[i], qmle_array[i])
    min_pen = binary_search_on_score_for_q_min_pen(observed[i], expected[i], delta, qmle_array[i])
    max_pen = binary_search_on_score_for_q_max_pen(observed[i], expected[i], delta, qmle_array[i])
    max_unpen = binary_search_on_score_for_q_max(observed[i], expected[i], qmle_array[i])
    intervals[i] = [min_unpen, min_pen, max_pen, max_unpen]


intervals = np.array(intervals)
q_values = np.unique(intervals.reshape(1, -1))
q_values

array([0.99999959, 0.99999961, 0.99999961, ..., 8.69730314, 9.57633293,
       9.57655575])

Search through filter to see if element in $S \backslash \Theta $ or $ \Theta \backslash S$

In [89]:
filter1 = poissons(expected)
filter2 = poissons(expected)